# My

In [34]:
import os 

import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split

import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer
# from nltk.corpus import wordnet as wn
from collections import defaultdict
import regex as re



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [92]:
root = "/content/drive/MyDrive/MTL782-Memes Classifier/dataminingmtl782/"
train_path = "train_images/train_images"
test_path = "test_images"

train_dt = pd.read_csv("/content/drive/MyDrive/MTL782-Memes Classifier/dataminingmtl782/train.csv")
test_dt = pd.read_csv("/content/drive/MyDrive/MTL782-Memes Classifier/dataminingmtl782/test.csv")
train_dt.head()

,ID,image id,text,label,label_num
0,1,image_2455.jpg,- It is not our fight - Are we not part of thi...,troll,2
1,2,image_3701.jpg,THAT'S THE DIFFERENCE BETWEEN YOU AND ME YOU...,none,0
2,3,image_4166.png,- WHAT DO THE TITANIC AND THE SIXTH SENSE HAVE...,none,0
3,4,448_image_batch_2.png,"""COME ON MAN, YOU KNOW THE THING.\r\nJUST ASK ...",troll,2
4,5,653_image_batch_2.png,"""Those who believe without reason cannot be co...",none,0


In [93]:
def get_files_labels(subpath,csv):
    """Gets paths images within a folder, and their labels"""
    img_paths, img_labels, texts = [], [], []
    df = pd.read_csv(root+csv)
    i=0
    for index, row in df.iterrows():
        try:
          img, text, label = row["image id"], row["text"], int(row["label_num"])
        except:
          img, text, label = row["image id"], row["text"], -1
        # print(img_,": ",text_)
        
        ## Change this for img....>>>
        ###if not (i in [858, 1986, 1990] and csv=="train.csv"):
        if True:
          img_paths.append(root+subpath+'/'+img.strip())
          img_labels.append(label)
          ###texts.append(preprocess_txt(text))
          ###texts.append(text)
          texts.append(text)
        i+=1
        #print('\r images: {} labels : {}'.format(len(img_paths), len(img_labels)) )
    return img_paths, img_labels, texts

In [94]:
files_, y_, texts_ = get_files_labels(train_path,"train.csv")
files_test, y_test, texts_test = get_files_labels(test_path,"test.csv")


In [ ]:
len(texts_)

1991

# Emotion Classification in texts using LSTM and Word2Vec

### Architecture: 
(X) Text -> Embedding (W2V pretrained on wikipedia articles) -> Deep Network (CNN 1D) -> Fully connected (Dense) -> Output Layer (Softmax) -> Emotion class (Y)

#### Embedding Layer
* Word Embedding is a representation of text where words that have the similar meaning have a similar representation. We will use 300 dimentional word vectors pre-trained on wikipedia articles. We can also train the w2v model with our data, however our dataset is quite small and trained word vectors might not be as good as using pretrained w2v.

#### Deep Network
* Though text data is one-dimensional, we can use 1D convolutional neural networks to extract features from our data. The result of each convolution will fire when a special pattern is detected. By varying the size of the kernels and concatenating their outputs, you’re allowing yourself to detect patterns of multiples sizes (2, 3, or 5 adjacent words). Patterns could be expressions like “I hate”, “very good” and therefore CNNs can identify them in the sentence regardless of their position. 

#### Fully Connected Layer
* The fully connected layer takes the deep representation from the RNN/LSTM/GRU and transforms it into the final output classes or class scores. This component is comprised of fully connected layers along with batch normalization and optionally dropout layers for regularization.

#### Output Layer
* Based on the problem at hand, this layer can have either Sigmoid for binary classification or Softmax for both binary and multi classification output.

## Workflow: 
1. Import Data
2. Prepare the input data
3. Import pre-trained W2V
4. Create Neural Network Pipeline
5. Train The Model
6. Evaluate results



👋  **Let's start** 

## 1. Import Data


In [35]:
import pandas as pd
import numpy as np

# text preprocessing
from nltk.tokenize import word_tokenize
import re

# plots and metrics
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# preparing input to our model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

# keras layers
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

Defining vector space dimension and fixed input size

In [36]:
# Number of labels: joy, anger, fear, sadness, neutral
num_classes = 5

# Number of dimensions for word embedding
embed_num_dims = 300

# Max input length (max number of words) 
max_seq_len = 500

class_names = ['joy', 'fear', 'anger', 'sadness', 'neutral']

Importing our training and testing datasets

In [37]:
data_train = pd.read_csv('data/data_train.csv', encoding='utf-8')
data_test = pd.read_csv('data/data_test.csv', encoding='utf-8')

X_train = data_train.Text
X_test = data_test.Text

y_train = data_train.Emotion
y_test = data_test.Emotion

data = data_train.append(data_test, ignore_index=True)

In [38]:
print(data.Emotion.value_counts())
data.head(6)

joy        2326
sadness    2317
anger      2259
neutral    2254
fear       2171
Name: Emotion, dtype: int64


,Emotion,Text
0,neutral,There are tons of other paintings that I thin...
1,sadness,"Yet the dog had grown old and less capable , a..."
2,fear,When I get into the tube or the train without ...
3,fear,This last may be a source of considerable disq...
4,anger,She disliked the intimacy he showed towards so...
5,sadness,When my family heard that my Mother's cousin w...


## 2. Prepare input data
To input the data to our NN Model we'll need some preprocessing:
1. Tokenize our texts and count unique tokens
2. Padding: each input (sentence or text) has to be of the same lenght
3. Labels have to be converted to integeres and categorized

Basic preprocessing and tokenization using nltk to double check that sentences are properly split into words.
We could also add stopword removal but steps like stemming or lemmatization are not needed since we are using word2vec and words with the same stem can have a different meaning

In [39]:
import nltk
nltk.download('punkt')
def clean_text(data):
    
    # remove hashtags and @usernames
    data = re.sub(r"(#[\d\w\.]+)", '', data)
    data = re.sub(r"(@[\d\w\.]+)", '', data)
    
    # tekenization using nltk
    data = word_tokenize(data)
    
    return data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


*Making things easier for keras tokenizer 🙃

In [40]:
texts = [' '.join(clean_text(text)) for text in data.Text]

texts_train = [' '.join(clean_text(text)) for text in X_train]
texts_test = [' '.join(clean_text(text)) for text in X_test]

In [41]:
print(texts_train[92])

a bit ? I 'm extremely annoyed that he did n't phone me when he promised me that he would ! He 's such a liar .


**Tokenization + fitting using keras**

In [42]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

sequence_train = tokenizer.texts_to_sequences(texts_train)
sequence_test = tokenizer.texts_to_sequences(texts_test)

index_of_words = tokenizer.word_index

# vacab size is number of unique words + reserved 0 index for padding
vocab_size = len(index_of_words) + 1

print('Number of unique words: {}'.format(len(index_of_words)))

Number of unique words: 12088


**Padding** -> each input has the same length

We defined maximun number of words for our texts and input size to our model has to be fixed - padding with zeros to keep the same input lenght (longest input in our dataset is ~250 words)

In [43]:
X_train_pad = pad_sequences(sequence_train, maxlen = max_seq_len )
X_test_pad = pad_sequences(sequence_test, maxlen = max_seq_len )

X_train_pad

array([[    0,     0,     0, ...,   119,    51,   345],
       [    0,     0,     0, ...,    37,   277,   154],
       [    0,     0,     0, ...,    16,     2,  1210],
       ...,
       [    0,     0,     0, ...,   876,     4,   909],
       [    0,     0,     0, ...,     1,     6,   117],
       [    0,     0,     0, ..., 10259,   173,    13]], dtype=int32)

**Categorize** labels: 

In [44]:
encoding = {
    'joy': 0,
    'fear': 1,
    'anger': 2,
    'sadness': 3,
    'neutral': 4
}

# Integer labels
y_train = [encoding[x] for x in data_train.Emotion]
y_test = [encoding[x] for x in data_test.Emotion]

In [45]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

y_train

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

## 2. Import pretrained word vectors

* Importing pretrained word2vec from file and creating embedding matrix
* We will later map each word in our corpus to existing word vector

In [ ]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]
    return embedding_matrix

You can download and import any pre-trained word embeddings. I will use 300 dimentional w2v pre-trained on wikipedia articles. Download fast text english vectors: https://fasttext.cc/docs/en/english-vectors.html

In [ ]:
import urllib.request
import zipfile
import os

fname = 'embeddings/wiki-news-300d-1M.vec'

if not os.path.isfile(fname):
    print('Downloading word vectors...')
    urllib.request.urlretrieve('https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip',
                              'wiki-news-300d-1M.vec.zip')
    print('Unzipping...')
    with zipfile.ZipFile('wiki-news-300d-1M.vec.zip', 'r') as zip_ref:
        zip_ref.extractall('embeddings')
    print('done.')
    
    os.remove('wiki-news-300d-1M.vec.zip')

In [ ]:
embedd_matrix = create_embedding_matrix(fname, index_of_words, embed_num_dims)
embedd_matrix.shape

Some of the words from our corpus were not included in the pre-trained word vectors. If we inspect those words we'll see that it's mostly spelling errors. It's also good to double check the noise in our data f.e different languages or tokenizer errors.

In [ ]:
# Inspect unseen words
new_words = 0

for word in index_of_words:
    entry = embedd_matrix[index_of_words[word]]
    if all(v == 0 for v in entry):
        new_words = new_words + 1

print('Words found in wiki vocab: ' + str(len(index_of_words) - new_words))
print('New words found: ' + str(new_words))

## 3. Create CNN Pipeline

### Embedding Layer

We will use pre-trained word vectors. We could also train our own embedding layer if we don't specify the pre-trained weights 

* **vocabulary size:** the maximum number of terms that are used to represent a text: e.g. if we set the size of the “vocabulary” to 1000 only the first thousand terms most frequent in the corpus will be considered (and the other terms will be ignored)
* **the maximum length:** of the texts (which must all be the same length)
* **size of embeddings:** basically, the more dimensions we have the more precise the semantics will be, but beyond a certain threshold we will lose the ability of the embedding to define a coherent and general enough semantic area
* **trainable:** True if you want to fine-tune them while training


In [ ]:
# Embedding layer before the actaul BLSTM 
embedd_layer = Embedding(vocab_size,
                         embed_num_dims,
                         input_length = max_seq_len,
                         weights = [embedd_matrix],
                         trainable=False)

### Model Pipeline
- the input is the first N words of each text (with proper padding)
- the first level creates embedding of words, using vocabulary with a certain dimension, and a given size of embeddings
- we will use 1D convolutional neural network to extract features from our data. The result of each convolution will fire when a special pattern is detected. By varying the size of the kernels and concatenating their outputs, you’re allowing yourself to detect patterns of multiples sizes (2, 3, or 5 adjacent words)
- the output level has a number of neurons equal to the classes of the problem and a “softmax” activation function

In [ ]:
# Convolution
kernel_size = 3
filters = 256

model = Sequential()
model.add(embedd_layer)
model.add(Conv1D(filters, kernel_size, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

## 3. Train the Model

In [ ]:
batch_size = 256
epochs = 6

hist = model.fit(X_train_pad, y_train, 
                 batch_size=batch_size,
                 epochs=epochs,
                 validation_data=(X_test_pad,y_test))

In [ ]:
# Accuracy plot
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# Loss plot
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

## 4. Evaluation

In [ ]:
predictions = model.predict(X_test_pad)
predictions = np.argmax(predictions, axis=1)
predictions = [class_names[pred] for pred in predictions]

In [ ]:
print("Accuracy: {:.2f}%".format(accuracy_score(data_test.Emotion, predictions) * 100))
print("\nF1 Score: {:.2f}".format(f1_score(data_test.Emotion, predictions, average='micro') * 100))

#### Plotting confusion Matrix:

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    '''
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    '''
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    fig, ax = plt.subplots()
    
    # Set size
    fig.set_size_inches(12.5, 7.5)
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    ax.grid(False)
    
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
print("\nF1 Score: {:.2f}".format(f1_score(data_test.Emotion, predictions, average='micro') * 100))

# Plot normalized confusion matrix
plot_confusion_matrix(data_test.Emotion, predictions, classes=class_names, normalize=True, title='Normalized confusion matrix')
plt.show()

#### Let's try other inputs:

In [ ]:
print('Message: {}\nPredicted: {}'.format(X_test[4], predictions[4]))

In [ ]:
import time

message = ['delivery was hour late and my pizza was cold!']

seq = tokenizer.texts_to_sequences(message)
padded = pad_sequences(seq, maxlen=max_seq_len)

start_time = time.time()
pred = model.predict(padded)

print('Message: ' + str(message))
print('predicted: {} ({:.2f} seconds)'.format(class_names[np.argmax(pred)], (time.time() - start_time)))

# Done
Save the model for later use 🙃 

In [ ]:
# creates a HDF5 file 'my_model.h5'
model.save('models/cnn_w2v.h5')

In [46]:
#$# !pip install ktrain

In [48]:
from keras.models import load_model
predictor = load_model('models/cnn_w2v.h5')

##import ktrain
##predictor = ktrain.load_predictor("models/bert_models")

In [49]:
files_, y_, texts_ = get_files_labels(train_path,"train.csv")
files_test, y_test, _texts_test = get_files_labels(test_path,"test.csv")

In [51]:
import time

message = ['''so sad that Steve Jobs died of Ligma who's steve Jobs? Ligma Balls''']

seq = tokenizer.texts_to_sequences(message)
padded = pad_sequences(seq, maxlen=max_seq_len)
##padded = message

start_time = time.time()
pred = predictor.predict(padded)

print('Message: ' + str(message))
print('predicted: {} ({:.2f} seconds)'.format(pred, (time.time() - start_time)))

Message: ["so sad that Steve Jobs died of Ligma who's steve Jobs? Ligma Balls"]
predicted: [[8.1978964e-05 3.1834701e-05 1.3791790e-04 9.9969125e-01 5.7124969e-05]] (0.04 seconds)


In [52]:
import time

seq = tokenizer.texts_to_sequences(texts_[0:50])
padded = pad_sequences(seq, maxlen=max_seq_len)

start_time = time.time()
pred_train = predictor.predict(padded)

pred_ = np.argmax(pred_train,axis=1)

print('Message: ' + str(message))
print('predicted: {} ({:.2f} seconds)'.format(pred_, (time.time() - start_time)))
print('gold: {} '.format(y_[:50]))

Message: ["so sad that Steve Jobs died of Ligma who's steve Jobs? Ligma Balls"]
predicted: [4 1 1 1 1 0 4 0 1 0 1 4 3 1 1 4 0 4 2 2 4 2 3 1 1 2 4 1 1 4 1 2 1 3 0 2 2
 4 1 1 2 4 2 2 1 4 1 1 4 0] (0.56 seconds)
gold: [2, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 2, 2, 1, 0, 1, 1, 0, 0, 0, 0, 2, 0, 1, 0, 2, 0, 1, 1, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 2, 1, 1, 1, 1] 


In [53]:
#print(*sorted(zip(pred_,y_[:50])),sep="\n")
print(pred[0])

[8.1978964e-05 3.1834701e-05 1.3791790e-04 9.9969125e-01 5.7124969e-05]


In [54]:
_text_train, _text_val, _y_train, _y_val = train_test_split(
    texts_, y_, test_size=0.1, random_state=42, stratify=y_, shuffle=True)
#X_test = np.column_stack([files_test,texts_test])
# X_train[0]

In [55]:
len(_text_train), len(_text_val), len(_y_train), len(_y_val)

(1791, 200, 1791, 200)

In [56]:
import time

seq = tokenizer.texts_to_sequences(_text_train)
padded = pad_sequences(seq, maxlen=max_seq_len)

#padded  = _text_train

start_time = time.time()
pred_train = predictor.predict(padded)

pred_ = np.argmax(pred_train,axis=1)

#print('Message: ' + str(message))
#print('predicted: {} ({:.2f} seconds)'.format(pred_, (time.time() - start_time)))
#print('gold: {} '.format(y_[:50]))

In [57]:
import time

seq = tokenizer.texts_to_sequences(_text_val)
padded = pad_sequences(seq, maxlen=max_seq_len)

start_time = time.time()
pred_val = predictor.predict(padded)

pred_ = np.argmax(pred_val,axis=1)

#print('Message: ' + str(message))
#print('predicted: {} ({:.2f} seconds)'.format(pred_, (time.time() - start_time)))
#print('gold: {} '.format(y_[:50]))

In [90]:
import pickle
pickle.dump(pred_train, open("__cnn_pred_train.pkl", 'wb'))
pickle.dump(pred_val, open("__cnn_pred_val.pkl", 'wb'))


In [82]:
from sklearn.svm import SVC,LinearSVC
from sklearn.tree import DecisionTreeClassifier
svc = DecisionTreeClassifier()
svc.fit(pred_train,_y_train)

DecisionTreeClassifier()

In [83]:
from sklearn.metrics import classification_report
print(classification_report(svc.predict(pred_train), _y_train))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00       547
           1       1.00      1.00      1.00       623
           2       0.99      1.00      0.99       621

    accuracy                           1.00      1791
   macro avg       1.00      1.00      1.00      1791
weighted avg       1.00      1.00      1.00      1791



In [89]:
from sklearn.metrics import classification_report
print(classification_report(svc.predict(pred_val[100:150]), _y_val[100:150]))

              precision    recall  f1-score   support

           0       0.38      0.33      0.35        18
           1       0.33      0.35      0.34        17
           2       0.44      0.47      0.45        15

    accuracy                           0.38        50
   macro avg       0.38      0.38      0.38        50
weighted avg       0.38      0.38      0.38        50



In [85]:
import pickle
pickle.dump(svc, open("nlptone__dt6_b.sav", 'wb'))

In [ ]:
len(pred_)

1791

In [ ]:
import collections
occurrences = collections.Counter(sorted(zip(pred_,_y_train)))


In [ ]:
l = collections.Counter(_y_train)
#l

In [ ]:
#occurrences

In [ ]:
#for key,val in occurrences.items():
#  print(key,val/l[key[1]])

In [ ]:
#m = {0:0,1:2,2:1,3:2,4:0}

In [ ]:
len(pred_)

200

In [ ]:
#pred_

In [ ]:
#for i in range(200):
#  pred_[i] = m[pred_[i]]

In [ ]:
#pred_

In [ ]:
svc.predict_proba(pred)

In [ ]:
import pickle
pickle.dump(svc, open("nlptone__dt2.sav", 'wb'))

In [ ]:
"""
import pickle
svc = pickle.load(open("nlptone__dt.sav", 'rb'))
"""

In [95]:
import time

seq = tokenizer.texts_to_sequences(_texts_test)
padded = pad_sequences(seq, maxlen=max_seq_len)

start_time = time.time()
pred_test = predictor.predict(padded)

#pred_ = np.argmax(_texts_test,axis=1)

#print('Message: ' + str(message))
#print('predicted: {} ({:.2f} seconds)'.format(pred_, (time.time() - start_time)))
#print('gold: {} '.format(y_[:50]))



In [96]:
pred_test.shape

(600, 5)

In [97]:
import pickle
pickle.dump(pred_test, open("__cnn_pred_test.pkl", 'wb'))


In [98]:
import pickle
pickle.dump(_y_train, open("__cnn_y_train.pkl", 'wb'))
pickle.dump(_y_val, open("__cnn_y_val.pkl", 'wb'))


In [ ]:
print(*svc.predict(pred), sep="\n")

In [ ]:
len(texts_test)

600

In [6]:
!cp -r "/content/drive/MyDrive/MTL782-Memes Classifier/bert_model" models/bert_models/